In [ ]:
import cv2
import mediapipe as mp
import pygetwindow as gw
import pyautogui

# Mediapipe setup
mp_hands = mp.solutions.hands
hands = mp_hands.Hands(min_detection_confidence=0.7, min_tracking_confidence=0.7)
mp_draw = mp.solutions.drawing_utils

# Capture webcam input
cap = cv2.VideoCapture(0)
cap.set(3, 1280)  # Width
cap.set(4, 720)   # Height

# Screen dimensions
screen_width, screen_height = pyautogui.size()

# State variables
selected_window = None
dragging = False
offset_x = 0
offset_y = 0


def map_to_screen(webcam_x, webcam_y, cam_width=1280, cam_height=720):
    """Map webcam coordinates to screen coordinates."""
    screen_x = int((webcam_x / cam_width) * screen_width)
    screen_y = int((webcam_y / cam_height) * screen_height)
    return screen_x, screen_y


while True:
    ret, frame = cap.read()
    if not ret:
        break

    frame = cv2.flip(frame, 1)  # Flip frame for a mirror effect
    rgb_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

    # Process the frame
    result = hands.process(rgb_frame)

    # Hand landmark tracking
    if result.multi_hand_landmarks:
        for hand_landmarks in result.multi_hand_landmarks:
            h, w, _ = frame.shape

            # Index finger and thumb tip
            index_finger = hand_landmarks.landmark[mp_hands.HandLandmark.INDEX_FINGER_TIP]
            thumb_tip = hand_landmarks.landmark[mp_hands.HandLandmark.THUMB_TIP]

            index_fingertip = (int(index_finger.x * w), int(index_finger.y * h))
            thumb_tip_coord = (int(thumb_tip.x * w), int(thumb_tip.y * h))

            # Draw landmarks
            mp_draw.draw_landmarks(frame, hand_landmarks, mp_hands.HAND_CONNECTIONS)

            # Calculate pinch distance
            pinch_distance = ((index_fingertip[0] - thumb_tip_coord[0]) ** 2 +
                              (index_fingertip[1] - thumb_tip_coord[1]) ** 2) ** 0.5

            # Pinch gesture detected
            if pinch_distance < 50:  # Adjust threshold for pinch detection
                screen_x, screen_y = map_to_screen(index_fingertip[0], index_fingertip[1])

                if not dragging:
                    # Detect window under the pinch
                    for window in gw.getWindowsWithTitle(''):
                        if window.topleft.x <= screen_x <= window.bottomright.x and \
                                window.topleft.y <= screen_y <= window.bottomright.y:
                            selected_window = window
                            offset_x = screen_x - selected_window.left
                            offset_y = screen_y - selected_window.top
                            dragging = True
                            break
                else:
                    # Move the selected window
                    if selected_window:
                        new_left = screen_x - offset_x
                        new_top = screen_y - offset_y
                        selected_window.moveTo(new_left, new_top)
            else:
                # Release the window when pinch is released
                dragging = False
                selected_window = None

    # Display the webcam frame
    cv2.imshow("Webcam", frame)

    # Exit on ESC key
    if cv2.waitKey(1) & 0xFF == 27:
        break

cap.release()
cv2.destroyAllWindows()



